# 1. 윤석열 스크랩

In [1]:
# 첫번째 페이지
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화

# ChromeDriver 경로 설정 및 Service 객체 생성
s = Service(r"C:\Users\junse\2024-2\python_onlinedata_analysis\chromedriver-win64\chromedriver.exe")  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성

# 조선일보 사이트 접속
url_Ch = "https://www.chosun.com/"
driver.get(url_Ch)

try:
    # 돋보기
    search_icon =  driver.find_element(By.XPATH, '//*[@id="nav-bar-left"]/div[2]')
    search_icon.click()
    
    # 검색창 대기
    search_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="nav-bar-left"]/div[2]/form/div/input'))
    )
    # 검색어 입력
    search_input.send_keys("윤석열")
    search_input.send_keys(Keys.RETURN)  # 엔터키로 검색

    # 다음 페이지 로딩 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div[1]/div[1]/div[1]/span'))
    )
    
except Exception as e:
    print("검색창에 접근할 수 없습니다:", e) 
    driver.quit()
    exit()
    
# 검색 기간 설정
try:
    relevance = driver.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[1]/div[2]/div[1]/span')
    relevance.click()
    
    period = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="main"]/div[1]/div[2]/div/div[1]/button/span'))
    )
    period.click()           # 기간 버튼 누르기

    direct_input = driver.find_element(By.XPATH, '//*[@id="direct"]/span')   ## 직접입력 누르기
    direct_input.click()

    ## 시작 시점
    year_start  = driver.find_element(By.XPATH, '//*[@id="year_2022"]')   ## 연도 누르기 (2022)
    year_start.click()   

    month_start  = driver.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[2]/div/div[1]/div/div[3]/div[1]/div[3]/ul[2]/li[2]')   ## 달 누르기 (02)
    month_start.click()
    
    day_start = driver.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[2]/div/div[1]/div/div[3]/div[1]/div[3]/ul[3]/li[15]')   ## 일 누르기 (15)
    day_start.click()  
    
    ## 끝 시점
    end_button = driver.find_element(By.XPATH, '//*[@id="end"]')   ## 끝 시점 창 누르기
    end_button.click()
    
    year_end  = driver.find_element(By.XPATH, '//*[@id="year_2022"]')   ## 연도 누르기 (2022)
    year_end.click()   

    month_end = driver.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[2]/div/div[1]/div/div[3]/div[1]/div[3]/ul[2]/li[3]')   ## 달 누르기 (03)
    month_end.click()
    
    day_end = driver.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[2]/div/div[1]/div/div[3]/div[1]/div[3]/ul[3]/li[8]')   ## 일 누르기 (08)
    day_end.click()  
    
    # 적용 누르기
    apply_icon = driver.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[2]/div/div[1]/div/div[3]/div[2]/button[1]')
    apply_icon.click()

    # 옵션 유지 누르기 
    keep_option = driver.find_element(By.XPATH, '//*[@id="main"]/div[3]/div/label')
    keep_option.click()
    
except Exception as e:
    print("검색 기간을 설정할 수 없습니다:", e)
    driver.quit()
    exit()
#-----------------------------------------------------------------------------------------------------
# 기사 데이터 크롤링
time.sleep(5)
titles, links, dates = [], [], [] # 제목, 링크, 작성일자를 저장할 리스트 생성
page_source = driver.page_source
soup = BeautifulSoup(page_source, "lxml")  # BeautifulSoup으로 HTML 파싱

articles = soup.find_all("div", class_="story-card__headline")
for article in articles:
        
    try:
        # 기사 제목
        title_tag = article.find("a", class_="text__link")
        if title_tag :
            titles.append(title_tag.get_text(strip=True))
            links.append(title_tag["href"])
        else:
            titles.append("제목없음")
            links.append("링크없음")
    except Exception as e: 
        print(f"헤드라인 데이터 추출 오류: {e}")
        
breadcrumb_articles = soup.find_all("div", class_="story-card__breadcrumb")
for breadcrumb in breadcrumb_articles:
    try:
        # 날짜 추출
        date_tag = breadcrumb.find_all("span")[-1]  # 마지막 <span> 태그에서 날짜 추출
        if date_tag:
            dates.append(date_tag.get_text(strip=True))
        else:
            dates.append("날짜 없음")
    except Exception as e:
        print(f"날짜 데이터 추출 오류: {e}")
    
#--------------------------------------------------------------------------------------------------------------
# 데이터 저장
try:
    df = pd.DataFrame({"제목": titles, "링크": links, "작성일자": dates})
    df = df.drop_duplicates()
    df.to_csv("윤석열_조선일보_기사목록.csv", index=False, encoding="utf-8-sig")
    print("데이터가 CSV 파일로 저장되었습니다.")
except Exception as e:
    print("CSV 저장 중 오류 발생:", e)

# 브라우저 종료
driver.quit()

데이터가 CSV 파일로 저장되었습니다.


In [15]:
## 이거 반복
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화

# ChromeDriver 경로 설정 및 Service 객체 생성
s = Service(r"C:\Users\junse\2024-2\python_onlinedata_analysis\chromedriver-win64\chromedriver.exe")  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성

# 조선일보 사이트 접속
for numb in range(2,51):
    url_Ch = f"https://www.chosun.com/nsearch/?query=%EC%9C%A4%EC%84%9D%EC%97%B4&page={numb}&siteid=&sort=0&date_period=direct&date_start=20220215&date_end=20220308&writer=&field=&emd_word=&expt_word=&opt_chk=true&app_check=0&website=www,chosun&category="
    driver.get(url_Ch)

#-----------------------------------------------------------------------------------------------------

    # 기사 데이터 크롤링
    time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "lxml")  # BeautifulSoup으로 HTML 파싱

    articles = soup.find_all("div", class_="story-card__headline")
    for article in articles:
        
        try:
            # 기사 제목, 링크
            title_tag = article.find("a", class_="text__link")
            if title_tag :
                titles.append(title_tag.get_text(strip=True))
                links.append(title_tag["href"])
            else:
                titles.append("제목없음")
                links.append("링크없음")
        except Exception as e: 
            print(f"헤드라인 데이터 추출 오류: {e}")
        
    breadcrumb_articles = soup.find_all("div", class_="story-card__breadcrumb")
    for breadcrumb in breadcrumb_articles:
        try:
            # 날짜 추출
            date_tag = breadcrumb.find_all("span")[-1]  # 마지막 <span> 태그에서 날짜 추출
            if date_tag:
                dates.append(date_tag.get_text(strip=True))
            else:
                dates.append("날짜 없음")
        except Exception as e:
            print(f"날짜 데이터 추출 오류: {e}")
    
#--------------------------------------------------------------------------------------------------------------
# 데이터 저장
try:
    df = pd.DataFrame({"제목": titles, "링크": links, "작성일자": dates})
    df = df.drop_duplicates()
    df.to_csv("윤석열_조선일보_기사목록.csv", index=False, encoding="utf-8-sig")
    print("데이터가 CSV 파일로 저장되었습니다.")
except Exception as e:
    print("CSV 저장 중 오류 발생:", e)

# 브라우저 종료
driver.quit()

데이터가 CSV 파일로 저장되었습니다.


In [1]:
## 이거 하나만
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화

# ChromeDriver 경로 설정 및 Service 객체 생성
s = Service(r"C:\Users\junse\2024-2\python_onlinedata_analysis\chromedriver-win64\chromedriver.exe")  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성
titles, links, dates = [], [], [] # 제목, 링크, 작성일자를 저장할 리스트 생성

# 조선일보 사이트 접속
for numb in range(1,21):
    url_Ch = f"https://www.chosun.com/nsearch/?query=%EC%9C%A4%EC%84%9D%EC%97%B4&page={numb}&siteid=&sort=0&date_period=direct&date_start=20220215&date_end=20220308&writer=&field=&emd_word=&expt_word=&opt_chk=true&app_check=0&website=www,chosun&category="
    driver.get(url_Ch)

#-----------------------------------------------------------------------------------------------------

    # 기사 데이터 크롤링
    time.sleep(3)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "lxml")  # BeautifulSoup으로 HTML 파싱

    articles = soup.find_all("div", class_="story-card__headline")
    for article in articles:
        
        try:
            # 기사 제목, 링크
            title_tag = article.find("a", class_="text__link")
            if title_tag :
                titles.append(title_tag.get_text(strip=True))
                links.append(title_tag["href"])
            else:
                titles.append("제목없음")
                links.append("링크없음")
        except Exception as e: 
            print(f"헤드라인 데이터 추출 오류: {e}")
        
    breadcrumb_articles = soup.find_all("div", class_="story-card__breadcrumb")
    for breadcrumb in breadcrumb_articles:
        try:
            # 날짜 추출
            date_tag = breadcrumb.find_all("span")[-1]  # 마지막 <span> 태그에서 날짜 추출
            if date_tag:
                dates.append(date_tag.get_text(strip=True))
            else:
                dates.append("날짜 없음")
        except Exception as e:
            print(f"날짜 데이터 추출 오류: {e}")
    
#--------------------------------------------------------------------------------------------------------------
# 데이터 저장
try:
    df = pd.DataFrame({"제목": titles, "링크": links, "작성일자": dates})
    df = df.drop_duplicates()
    df.to_csv("윤석열_조선일보_기사목록200.csv", index=False, encoding="utf-8-sig")
    print("데이터가 CSV 파일로 저장되었습니다.")
except Exception as e:
    print("CSV 저장 중 오류 발생:", e)

# 브라우저 종료
driver.quit()

데이터가 CSV 파일로 저장되었습니다.


# 2. 이재명 스크랩

In [22]:
# 첫번째 페이지
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화

# ChromeDriver 경로 설정 및 Service 객체 생성
s = Service(r"C:\Users\junse\2024-2\python_onlinedata_analysis\chromedriver-win64\chromedriver.exe")  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성

# 조선일보 사이트 접속
url_Ch = "https://www.chosun.com/"
driver.get(url_Ch)

try:
    # 돋보기
    search_icon =  driver.find_element(By.XPATH, '//*[@id="nav-bar-left"]/div[2]')
    search_icon.click()
    
    # 검색창 대기
    search_input = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="nav-bar-left"]/div[2]/form/div/input'))
    )
    # 검색어 입력
    search_input.send_keys("이재명")
    search_input.send_keys(Keys.RETURN)  # 엔터키로 검색

    # 다음 페이지 로딩 대기
    WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//*[@id="main"]/div[1]/div[1]/div[1]/span'))
    )
    
except Exception as e:
    print("검색창에 접근할 수 없습니다:", e) 
    driver.quit()
    exit()
    
# 검색 기간 설정
try:
    relevance = driver.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[1]/div[2]/div[1]/span')
    relevance.click()
    
    period = WebDriverWait(driver, 10).until(
        EC.element_to_be_clickable((By.XPATH, '//*[@id="main"]/div[1]/div[2]/div/div[1]/button/span'))
    )
    period.click()           # 기간 버튼 누르기

    direct_input = driver.find_element(By.XPATH, '//*[@id="direct"]/span')   ## 직접입력 누르기
    direct_input.click()

    ## 시작 시점
    year_start  = driver.find_element(By.XPATH, '//*[@id="year_2022"]')   ## 연도 누르기 (2022)
    year_start.click()   

    month_start  = driver.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[2]/div/div[1]/div/div[3]/div[1]/div[3]/ul[2]/li[2]')   ## 달 누르기 (02)
    month_start.click()
    
    day_start = driver.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[2]/div/div[1]/div/div[3]/div[1]/div[3]/ul[3]/li[15]')   ## 일 누르기 (15)
    day_start.click()  
    
    ## 끝 시점
    end_button = driver.find_element(By.XPATH, '//*[@id="end"]')   ## 끝 시점 창 누르기
    end_button.click()
    
    year_end  = driver.find_element(By.XPATH, '//*[@id="year_2022"]')   ## 연도 누르기 (2022)
    year_end.click()   

    month_end = driver.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[2]/div/div[1]/div/div[3]/div[1]/div[3]/ul[2]/li[3]')   ## 달 누르기 (03)
    month_end.click()
    
    day_end = driver.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[2]/div/div[1]/div/div[3]/div[1]/div[3]/ul[3]/li[8]')   ## 일 누르기 (08)
    day_end.click()  
    
    # 적용 누르기
    apply_icon = driver.find_element(By.XPATH, '//*[@id="main"]/div[1]/div[2]/div/div[1]/div/div[3]/div[2]/button[1]')
    apply_icon.click()

    # 옵션 유지 누르기 
    keep_option = driver.find_element(By.XPATH, '//*[@id="main"]/div[3]/div/label')
    keep_option.click()
    
except Exception as e:
    print("검색 기간을 설정할 수 없습니다:", e)
    driver.quit()
    exit()
#-----------------------------------------------------------------------------------------------------
# 기사 데이터 크롤링
time.sleep(5)
titles, links, dates = [], [], [] # 제목, 링크, 작성일자를 저장할 리스트 생성
page_source = driver.page_source
soup = BeautifulSoup(page_source, "lxml")  # BeautifulSoup으로 HTML 파싱

articles = soup.find_all("div", class_="story-card__headline")
for article in articles:
        
    try:
        # 기사 제목
        title_tag = article.find("a", class_="text__link")
        if title_tag :
            titles.append(title_tag.get_text(strip=True))
            links.append(title_tag["href"])
        else:
            titles.append("제목없음")
            links.append("링크없음")
    except Exception as e: 
        print(f"헤드라인 데이터 추출 오류: {e}")
        
breadcrumb_articles = soup.find_all("div", class_="story-card__breadcrumb")
for breadcrumb in breadcrumb_articles:
    try:
        # 날짜 추출
        date_tag = breadcrumb.find_all("span")[-1]  # 마지막 <span> 태그에서 날짜 추출
        if date_tag:
            dates.append(date_tag.get_text(strip=True))
        else:
            dates.append("날짜 없음")
    except Exception as e:
        print(f"날짜 데이터 추출 오류: {e}")
    
#--------------------------------------------------------------------------------------------------------------
# 데이터 저장
try:
    df = pd.DataFrame({"제목": titles, "링크": links, "작성일자": dates})
    df = df.drop_duplicates()
    df.to_csv("이재명_조선일보_기사목록.csv", index=False, encoding="utf-8-sig")
    print("데이터가 CSV 파일로 저장되었습니다.")
except Exception as e:
    print("CSV 저장 중 오류 발생:", e)

# 브라우저 종료
driver.quit()

데이터가 CSV 파일로 저장되었습니다.


In [24]:
## 이거 반복
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화

# ChromeDriver 경로 설정 및 Service 객체 생성
s = Service(r"C:\Users\junse\2024-2\python_onlinedata_analysis\chromedriver-win64\chromedriver.exe")  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성

# 조선일보 사이트 접속
for numb in range(2,51):
    url_Ch = f"https://www.chosun.com/nsearch/?query=%EC%9D%B4%EC%9E%AC%EB%AA%85&page={numb}&siteid=&sort=0&date_period=direct&date_start=20220215&date_end=20220308&writer=&field=&emd_word=&expt_word=&opt_chk=true&app_check=0&website=www,chosun&category="
    driver.get(url_Ch)

#-----------------------------------------------------------------------------------------------------

    # 기사 데이터 크롤링
    time.sleep(5)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "lxml")  # BeautifulSoup으로 HTML 파싱

    articles = soup.find_all("div", class_="story-card__headline")
    for article in articles:
        
        try:
            # 기사 제목, 링크
            title_tag = article.find("a", class_="text__link")
            if title_tag :
                titles.append(title_tag.get_text(strip=True))
                links.append(title_tag["href"])
            else:
                titles.append("제목없음")
                links.append("링크없음")
        except Exception as e: 
            print(f"헤드라인 데이터 추출 오류: {e}")
        
    breadcrumb_articles = soup.find_all("div", class_="story-card__breadcrumb")
    for breadcrumb in breadcrumb_articles:
        try:
            # 날짜 추출
            date_tag = breadcrumb.find_all("span")[-1]  # 마지막 <span> 태그에서 날짜 추출
            if date_tag:
                dates.append(date_tag.get_text(strip=True))
            else:
                dates.append("날짜 없음")
        except Exception as e:
            print(f"날짜 데이터 추출 오류: {e}")
    
#--------------------------------------------------------------------------------------------------------------
# 데이터 저장
try:
    df = pd.DataFrame({"제목": titles, "링크": links, "작성일자": dates})
    df = df.drop_duplicates()
    df.to_csv("이재명_조선일보_기사목록.csv", index=False, encoding="utf-8-sig")
    print("데이터가 CSV 파일로 저장되었습니다.")
except Exception as e:
    print("CSV 저장 중 오류 발생:", e)

# 브라우저 종료
driver.quit()

데이터가 CSV 파일로 저장되었습니다.


In [3]:
## 이거 하나만
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import time

# Chrome 옵션 설정
chrome_options = Options()
chrome_options.add_experimental_option("detach", True)
chrome_options.add_argument("--disable-notifications")  # 알림 비활성화
chrome_options.add_argument("--start-maximized")  # 브라우저 최대화
chrome_options.add_argument("--disable-popup-blocking")  # 팝업 차단 비활성화

# ChromeDriver 경로 설정 및 Service 객체 생성
s = Service(r"C:\Users\junse\2024-2\python_onlinedata_analysis\chromedriver-win64\chromedriver.exe")  # ChromeDriver 실행 파일 경로
driver = webdriver.Chrome(service=s, options=chrome_options)  # WebDriver 객체 생성
titles, links, dates = [], [], [] # 제목, 링크, 작성일자를 저장할 리스트 생성

# 조선일보 사이트 접속
for numb in range(1,21):
    url_Ch = f"https://www.chosun.com/nsearch/?query=%EC%9D%B4%EC%9E%AC%EB%AA%85&page={numb}&siteid=&sort=0&date_period=direct&date_start=20220215&date_end=20220308&writer=&field=&emd_word=&expt_word=&opt_chk=true&app_check=0&website=www,chosun&category="
    driver.get(url_Ch)

#-----------------------------------------------------------------------------------------------------

    # 기사 데이터 크롤링
    time.sleep(3)
    page_source = driver.page_source
    soup = BeautifulSoup(page_source, "lxml")  # BeautifulSoup으로 HTML 파싱

    articles = soup.find_all("div", class_="story-card__headline")
    for article in articles:
        
        try:
            # 기사 제목, 링크
            title_tag = article.find("a", class_="text__link")
            if title_tag :
                titles.append(title_tag.get_text(strip=True))
                links.append(title_tag["href"])
            else:
                titles.append("제목없음")
                links.append("링크없음")
        except Exception as e: 
            print(f"헤드라인 데이터 추출 오류: {e}")
        
    breadcrumb_articles = soup.find_all("div", class_="story-card__breadcrumb")
    for breadcrumb in breadcrumb_articles:
        try:
            # 날짜 추출
            date_tag = breadcrumb.find_all("span")[-1]  # 마지막 <span> 태그에서 날짜 추출
            if date_tag:
                dates.append(date_tag.get_text(strip=True))
            else:
                dates.append("날짜 없음")
        except Exception as e:
            print(f"날짜 데이터 추출 오류: {e}")
    
#--------------------------------------------------------------------------------------------------------------
# 데이터 저장
try:
    df = pd.DataFrame({"제목": titles, "링크": links, "작성일자": dates})
    df = df.drop_duplicates()
    df.to_csv("이재명_조선일보_기사목록200.csv", index=False, encoding="utf-8-sig")
    print("데이터가 CSV 파일로 저장되었습니다.")
except Exception as e:
    print("CSV 저장 중 오류 발생:", e)

# 브라우저 종료
driver.quit()

데이터가 CSV 파일로 저장되었습니다.
